In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8MB 26kB/s 
     |████████████████████████████████| 6.6MB 17.8MB/s 
  Found existing installation: torch 1.5.1+cu101
    Uninstalling torch-1.5.1+cu101:
      Successfully uninstalled torch-1.5.1+cu101
  Found existing installation: torchvision 0.6.1+cu101
    Uninstalling torchvision-0.6.1+cu101:
      Successfully uninstalled torchvision-0.6.1+cu101
     |████████████████████████████████| 276kB 5.0MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44074 sha256=d32724a840312370bd0289aeb6051b605119290d1fc825739db90ba2b5ba9f88
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# install detectron2:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html
     |████████████████████████████████| 6.2MB 659kB/s 
  Created wheel for fvcore: filename=fvcore-0.1.1.post20200704-cp36-none-any.whl size=41894 sha256=c3a3dd8aee48f9823c282ba6e17890b3836709fe1ad9aa0dab470ae3b66bb559
  Stored in directory: /root/.cache/pip/wheels/fb/d2/8e/b6d0f19811e77dabff1ebed6605ce2b59ee9f487079b434c8c
Successfully built fvcore


In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [ ]:
# !unzip '/content/drive/My Drive/new hope/train.zip'
# !unzip '/content/drive/My Drive/new hope/test.zip'

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("cancer_train", {}, "/content/drive/My Drive/new hope/train.json", "/content/train")
MetadataCatalog.get("cancer_train").set(thing_classes=["tumor"])
cancer_matadata = MetadataCatalog.get("cancer_train")
print(cancer_matadata)

Metadata(evaluator_type='coco', image_root='/content/train', json_file='/content/drive/My Drive/new hope/train.json', name='cancer_train', thing_classes=['tumor'])


In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("cancer_test1", {}, "/content/drive/My Drive/new hope/test.json", "/content/test")
MetadataCatalog.get("cancer_test1").set(thing_classes=["tumor"])
cancer_matadata = MetadataCatalog.get("cancer_test1")
print(cancer_matadata)


Metadata(evaluator_type='coco', image_root='/content/test', json_file='/content/drive/My Drive/new hope/test.json', name='cancer_test1', thing_classes=['tumor'])


In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("cancer_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
# cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/mask_rcnn_X_101_32x8d_FPN_3x/139653917/model_final_2d9806.pkl.pkl" # Let training initialize from model zoo
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = '/content/drive/My Drive/new hope/model_0049999.pth'
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.000025
cfg.SOLVER.MAX_ITER = 70000   
# cfg.INPUT.MIN_SIZE_TRAIN = (1500,1500)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 1024
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (cancer)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=True)
trainer.train()

[07/08 08:50:10 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

In [ ]:
#10000 Iterations
cfg.MODEL.WEIGHTS = '/content/drive/My Drive/new hope/model_0049999.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2  # set the testing threshold for this model
cfg.DATASETS.TEST = ("cancer_train", )
predictor = DefaultPredictor(cfg)

In [ ]:
'''
Finding True Positive, False Negative samples in train dataset.
'''


from detectron2.utils.visualizer import ColorMode
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode
import math  
import csv
from tqdm import tqdm

dataset_dicts = DatasetCatalog.get("cancer_train")
cancer_metadata = MetadataCatalog.get("cancer_train")

TP = 0
FN = 0
FP = 0
total = 0
true_count = 0
false_count = 0

for d in tqdm(dataset_dicts): 

        bboxs = []
        bbox_center_x = []
        bbox_center_y = []

        predicted_centers_x = []
        predicted_centers_y = []

        im = cv2.imread(d['file_name'])
        outputs = predictor(im)

        for i in range(len(d['annotations'])):
            bboxs.append(d['annotations'][i]['bbox'])

        total = total + len(bboxs)

        for bbox in bboxs:
            bbox_center_x.append((bbox[0]+bbox[0]+bbox[2])/2)
            bbox_center_y.append((bbox[1]+bbox[1]+bbox[3])/2)

        predicted_bboxs = outputs["instances"].pred_boxes.to("cpu").tensor.numpy()
        
        for predicted_bbox in predicted_bboxs:
            predicted_centers_x.append((predicted_bbox[0]+predicted_bbox[2])/2)
            predicted_centers_y.append((predicted_bbox[1]+predicted_bbox[3])/2)  

        i = 0
        while i < len(predicted_centers_x):
            j = 0
            while j < len(predicted_centers_x):
                if i == j:
                    j += 1
                    pass
                else:
                    if (math.sqrt( (predicted_centers_y[j] - predicted_centers_y[i])**2 + 
                                        (predicted_centers_x[j] - predicted_centers_x[i])**2 ) ) <= 40:

                        predicted_centers_x = np.delete(predicted_centers_x,j)
                        predicted_centers_y = np.delete(predicted_centers_y,j)

                    else:
                        j += 1

            i += 1

        temp_bbox_x = bbox_center_x
        temp_bbox_y = bbox_center_y


        for m in range(len(predicted_centers_x)):
            for n in range(len(temp_bbox_x)):

                if (math.sqrt( (float(temp_bbox_x[n]) - (predicted_centers_x[m]) )**2 + 
                                (float(temp_bbox_y[n]) - (predicted_centers_y[m]) )**2 ) ) <= 40:
                                

                    # cv2_imshow(im[int(predicted_centers_y[n]-64):int(predicted_centers_y[n]+64), int(predicted_centers_x[n]-64):int(predicted_centers_x[n]+64)])
                    # try:
                    #     cv2.imwrite('/content/train_tp/tp_'+str(true_count)+'.jpg', im[int(predicted_centers_y[m]-32):int(predicted_centers_y[m]+32), int(predicted_centers_x[m]-32):int(predicted_centers_x[m]+32)])
                    # except Exception as e:
                    #     print(e)
                    # true_count = true_count+1
                    TP += 1
                    break
                if n == ( len(temp_bbox_x)-1 ):                 
                    FP += 1
                    # cv2_imshow(im[int(predicted_centers_y[m]-64):int(predicted_centers_y[m]+64), int(predicted_centers_x[m]-64):int(predicted_centers_x[m]+64)])
                    try:
                        cv2.imwrite('/content/train_fp/fp_'+str(false_count)+'.jpg', im[int(predicted_centers_y[m]-32):int(predicted_centers_y[m]+32), int(predicted_centers_x[m]-32):int(predicted_centers_x[m]+32)])
                    except Exception as e:
                        print(e)                    
                    false_count = false_count+1

print(FP)
print(TP)
print(total)


Precision = TP/(TP+FP)
Recall = TP/(total)

f_score = (2 * Precision * Recall) / (Precision + Recall)

print(Precision)
print(Recall)
print(f_score)

In [ ]:
'''
Finding True Positive, False Negative samples in train dataset.
'''


from detectron2.utils.visualizer import ColorMode
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode
import math  
import csv
from tqdm import tqdm

dataset_dicts = DatasetCatalog.get("cancer_test1")
cancer_metadata = MetadataCatalog.get("cancer_test1")

TP = 0
FN = 0
FP = 0
total = 0
true_count = 0
false_count = 0

for d in tqdm(dataset_dicts): 

        bboxs = []
        bbox_center_x = []
        bbox_center_y = []

        predicted_centers_x = []
        predicted_centers_y = []

        im = cv2.imread(d['file_name'])
        outputs = predictor(im)

        for i in range(len(d['annotations'])):
            bboxs.append(d['annotations'][i]['bbox'])

        total = total + len(bboxs)

        for bbox in bboxs:
            bbox_center_x.append((bbox[0]+bbox[0]+bbox[2])/2)
            bbox_center_y.append((bbox[1]+bbox[1]+bbox[3])/2)

        predicted_bboxs = outputs["instances"].pred_boxes.to("cpu").tensor.numpy()
        
        for predicted_bbox in predicted_bboxs:
            predicted_centers_x.append((predicted_bbox[0]+predicted_bbox[2])/2)
            predicted_centers_y.append((predicted_bbox[1]+predicted_bbox[3])/2)  

        i = 0
        while i < len(predicted_centers_x):
            j = 0
            while j < len(predicted_centers_x):
                if i == j:
                    j += 1
                    pass
                else:
                    if (math.sqrt( (predicted_centers_y[j] - predicted_centers_y[i])**2 + 
                                        (predicted_centers_x[j] - predicted_centers_x[i])**2 ) ) <= 30:

                        predicted_centers_x = np.delete(predicted_centers_x,j)
                        predicted_centers_y = np.delete(predicted_centers_y,j)

                    else:
                        j += 1

            i += 1

        temp_bbox_x = bbox_center_x
        temp_bbox_y = bbox_center_y


        for m in range(len(predicted_centers_x)):
            for n in range(len(temp_bbox_x)):

                if (math.sqrt( (float(temp_bbox_x[n]) - (predicted_centers_x[m]) )**2 + 
                                (float(temp_bbox_y[n]) - (predicted_centers_y[m]) )**2 ) ) <= 30:
                                

                    # cv2_imshow(im[int(predicted_centers_y[n]-64):int(predicted_centers_y[n]+64), int(predicted_centers_x[n]-64):int(predicted_centers_x[n]+64)])
                    try:
                        cv2.imwrite('/content/test_tp0.2/tp_'+str(true_count)+'.jpg', im[int(predicted_centers_y[m]-32):int(predicted_centers_y[m]+32), int(predicted_centers_x[m]-32):int(predicted_centers_x[m]+32)])
                    except Exception as e:
                        print(e)
                    true_count = true_count+1
                    TP += 1
                    break
                if n == ( len(temp_bbox_x)-1 ):                 
                    FP += 1
                    # cv2_imshow(im[int(predicted_centers_y[m]-64):int(predicted_centers_y[m]+64), int(predicted_centers_x[m]-64):int(predicted_centers_x[m]+64)])
                    try:
                        cv2.imwrite('/content/test_fp0.2/fp_'+str(false_count)+'.jpg', im[int(predicted_centers_y[m]-32):int(predicted_centers_y[m]+32), int(predicted_centers_x[m]-32):int(predicted_centers_x[m]+32)])
                    except Exception as e:
                        print(e)                    
                    false_count = false_count+1

print(FP)
print(TP)
print(total)


Precision = TP/(TP+FP)
Recall = TP/(total)

f_score = (2 * Precision * Recall) / (Precision + Recall)

print(Precision)
print(Recall)
print(f_score)

[07/08 09:04:26 d2.data.datasets.coco]: Loaded 155 images in COCO format from /content/drive/My Drive/new hope/test.json


  1%|          | 1/155 [00:00<01:06,  2.31it/s]

OpenCV(4.1.2) /io/opencv/modules/imgcodecs/src/loadsave.cpp:715: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'



 32%|███▏      | 49/155 [00:14<00:29,  3.63it/s]

OpenCV(4.1.2) /io/opencv/modules/imgcodecs/src/loadsave.cpp:715: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'



...

OpenCV(4.1.2) /io/opencv/modules/imgcodecs/src/loadsave.cpp:715: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'



 52%|█████▏    | 81/155 [00:23<00:20,  3.63it/s]

OpenCV(4.1.2) /io/opencv/modules/imgcodecs/src/loadsave.cpp:715: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'



100%|██████████| 155/155 [00:56<00:00,  2.74it/s]

217
324
487
0.5988909426987061
0.6652977412731006
0.6303501945525293
